## Interview Assistance with ChatGPT
### This notebook is used to test APIs and to build a prove of concept

The soundcard library uses pulseaudio and must be used on your local machine.  

In [2]:
import soundcard as sc
import soundfile as sf
from datetime import datetime
import openai

openai.api_key = "YOUR_OPENAI_API_KEY"

The duration of the recording is a constant for testing. In pratical, this should be dynamic using a good implementation method.  
  
According to the Nyquist-Shannon theorem, the sample rate should be at least two times the highest frequency you intend to capture.  
By using the standard sample rate of 44.1 kHz, you’ll perfectly record frequencies slightly higher than 20,000 Hz, which is the highest frequency level humans can hear.  

In [14]:
# Set the duration of the recording in seconds
RECORD_SECONDS = 5
# Set the sample rate
SR = 44100

Loopback enables allows to record sounds produced by our system as if they were being redirected into the microphone.  
In other words, a virtual microphones that record the output of a speaker.

In [4]:
# get a list of all loopback devices (devices that record system audio):
lb_devices = sc.all_microphones(include_loopback=True)

In [30]:
#print out the avaiable loopback devices
for i in range(len(lb_devices)):
    try:
        print(f"{i}: {lb_devices[i].name}")
    except Exception as e:
        print(e)

0: MSI G241 (HD Audio Driver for Display Audio)
1: AI Noise-cancelling Output (ASUS Utility)
2: Headphones (Realtek(R) Audio)
3: Speakers (Realtek(R) Audio)
4: Microphone (Realtek(R) Audio)
5: Microphone Array (Realtek(R) Audio)
6: AI Noise-cancelling Input (ASUS Utility)


After testing several different loopback devices, some does not work as intended and will stop recording with no error message.  
Try choosing Headphones (Realtek(R) Audio).  

In [31]:
# select the correct loopback device
lb_device = lb_devices[2]

In [34]:
# recording
print("Recording system audio...")
data = lb_device.record(numframes = RECORD_SECONDS * SR, samplerate= SR)
print("Done...")

Recording system audio...
Done...


In [35]:
# Set the filename using the date time
now = datetime.now().strftime("%m-%d_%H-%M")
file_name = f"q_{now}.wav"
# write the recorded data to a WAV file
sf.write(file_name, data, SR)  

To open the audo file you must download ffmepg.  
download: https://ffmpeg.org/download.html  
how to: https://phoenixnap.com/kb/ffmpeg-windows

Other functionalities and implementations of the whisper API is worth reading.  
whisper github: https://github.com/openai/whisper  
  
sorce of sample interview question: https://www.youtube.com/watch?v=GJdiM-muYqc

In [ ]:
audio_file = open(file_name, "rb")
# audio_file = open("sample_interview_question.wav", "rb")
# transcribe the auido file with whisper API
transcript = openai.Audio.translate("whisper-1", audio_file)
print(transcript["text"])

Which system prompt to use to get better results. Or are there other ways to optimize the response from the models?

In [ ]:
# prompt the model with the transcribed text and get the responce
responce = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "You are in a job or technical interview. Provide bullet point answers with a short explaination of reasoning."},
        {"role": "user", "content": transcript["text"]},
    ]
)
print(responce["choices"][0]["message"]["content"])

### Useful resources
Whisper API - Python and Gradio Tutorial: https://www.youtube.com/watch?v=Si0vFx_dJ5Y
